In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
pd.set_option('display.max_columns', 500)

In [3]:
df = pd.read_csv('heart_disease_indicators.csv')
df.head()

,HeartDisease/Attack,AgeGroup,GeneralHealthStatus,PhysicalHealthStatus,MentalHealthStatus,LastCheckup,HighBP,HighCholestrol,DiagnosedAsthma,Sex,MaritalStatus,Education,HomeStatus,VETERAN3,EmploymentStatus,ChildrenAtHome,IncomeCategory,Weight,Height,AlchoholDrinksPerDaY,FruitJuicesPerDay,BMI,FruitsEatenPerDay,VegetablesEatenPerDay,SmokingHabits,PhysicalActivity,StrengthActivty,_RFSEAT2
0,0.0,5.0,0.0,15.0,18.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,0.0,8.0,0.0,2.0,127.0,1.8,0.0,1.700000e+01,40.2,0.0,2.0,3.0,4.0,0.0,1.0
1,0.0,4.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,2.0,6.0,1.0,0.0,3.0,0.0,1.0,74.8,1.7,0.0,7.000000e+00,25.1,0.0,1.0,1.0,2.0,0.0,0.0
2,0.0,5.0,0.0,30.0,30.0,1.0,1.0,1.0,0.0,2.0,1.0,4.0,1.0,0.0,8.0,1.0,5.0,81.6,1.7,0.0,5.397605e-79,28.2,1.0,0.0,4.0,4.0,0.0,1.0
3,0.0,6.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,3.0,3.0,1.0,0.0,2.0,0.0,4.0,65.8,1.6,0.0,1.000000e+02,26.5,2.0,6.0,4.0,1.0,0.0,1.0
4,0.0,6.0,1.0,0.0,3.0,1.0,1.0,1.0,0.0,2.0,3.0,5.0,1.0,0.0,7.0,0.0,2.0,67.1,1.7,10.0,1.000000e+02,23.9,1.0,1.0,4.0,1.0,0.0,1.0


In [8]:
df.isna().sum()

HeartDisease/Attack          0
AgeGroup                     0
GeneralHealthStatus          0
PhysicalHealthStatus         0
MentalHealthStatus           0
LastCheckup                  1
HighBP                       0
HighCholestrol           36927
DiagnosedAsthma              0
Sex                          0
MaritalStatus                0
Education                    0
HomeStatus                   0
VETERAN3                     1
EmploymentStatus             0
ChildrenAtHome               0
IncomeCategory               0
Weight                    9303
Height                    2244
AlchoholDrinksPerDaY         0
FruitJuicesPerDay         3627
BMI                      12171
FruitsEatenPerDay         5133
VegetablesEatenPerDay     6175
SmokingHabits                0
PhysicalActivity             0
StrengthActivty           1317
_RFSEAT2                     0
dtype: int64